## Machine preparation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer, QuantumCircuit
from qiskit.opflow import Z, I, StateFn
from qiskit.utils import QuantumInstance
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap, ZFeatureMap, PauliFeatureMap
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, ADAM, SPSA, SLSQP


import pandas as pd

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from qiskit_machine_learning.algorithms.objective_functions import OneHotObjectiveFunction
from qiskit_machine_learning.neural_networks import TwoLayerQNN, CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR

from typing import Union

from qiskit_machine_learning.exceptions import QiskitMachineLearningError

from IPython.display import clear_output

from sklearn.feature_extraction import DictVectorizer

help("qiskit_machine_learning")

%matplotlib inline

Help on package qiskit_machine_learning:

NAME
    qiskit_machine_learning

DESCRIPTION
    Qiskit's Machine Learning module (:mod:`qiskit_machine_learning`)
    
    .. currentmodule:: qiskit_machine_learning
    
    This is the Qiskit`s machine learning module. There is an initial set of function here that
    will be built out over time. At present it has sample sets that can be used with
    :mod:`~qiskit_machine_learning.algorithms.classifiers` and circuits used in machine learning
    applications.
    
    .. autosummary::
       :toctree: ../stubs/
       :nosignatures:
    
        QiskitMachineLearningError
    
    Submodules
    
    .. autosummary::
       :toctree:
    
       algorithms
       circuit.library
       connectors
       datasets
       kernels
       neural_networks
       utils

PACKAGE CONTENTS
    algorithms (package)
    circuit (package)
    connectors (package)
    datasets (package)
    deprecation
    exceptions
    kernels (package)
    neural_net

In [2]:
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns 
    specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(
        vec.fit_transform(df[cols].to_dict('records')) \
        .toarray()
    )
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df


In [3]:
from qiskit import IBMQ
from qiskit.utils import QuantumInstance

use_real_device = true

if use_real_device:
    IBMQ.enable_account("PUT_IBMQ_TOKEN_HERE", )
    provider = IBMQ.get_provider(hub='ibm-q')
    backend = provider.get_backend('ibmq_lima')
    quantum_instance = QuantumInstance(backend, shots=1024)
else:
    quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator'), shots=1024)

In [4]:
objective_func_vals = []
weights_array = []
count = []
def callback_graph(weights, obj_func_eval):
    objective_func_vals.append(obj_func_eval)
    count.append(0)
    weights_array.append(weights)
    f = open("objective_func_vals.txt", "a")
    f.write(str(obj_func_eval))
    f.write("\n")
    f.close()
    
    f = open("weights_array.txt", "a")
    f.write(str(weights))
    f.write("\n")
    f.close()

In [5]:
RANDOM_STATE = 37
num_inputs = 4 # number of qubits
class_labels = ['Iris-setosa', 'Iris-versicolor','Iris-virginica']

In [6]:
def normalizeData(DATA_PATH = "iris.data"):
    """
    Normalizes the data
    """
    # Reads the data
    data = pd.read_csv(DATA_PATH)
#     print(data)
    data = shuffle(data, random_state=RANDOM_STATE)
    
    data = encode_onehot(data, cols=['class'])
#     print(data)
    # Split into data and result
    parameters = [
        'sepal_length', 
        'sepal_width', 
        'petal_length', 
        'petal_width'
    ]
    classes = [
        'class=Iris-setosa', 
        'class=Iris-versicolor', 
        'class=Iris-virginica'
    ]
    X, Y = data[parameters].values, data[classes].values
    
#     print(X)
#     print(Y)
    
    # normalize the data
    X = normalize(X)
    split = train_test_split(
        X, 
        Y, 
        test_size=0.3, 
        random_state=RANDOM_STATE
    )
    X_train, X_test, Y_train, Y_test = split
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = normalizeData()
print(Y_train)

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0.

/Library/Python/3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
output_shape = len(class_labels)

def interpretation(x):
    return f"{x:b}".count("1") % output_shape

def construct_circuit(
    feature_map, 
    ansatz, 
    quantum_instance, 
    number_of_inputs
):
    qc = QuantumCircuit(number_of_inputs)
    qc.append(feature_map, range(number_of_inputs))
    qc.append(ansatz, range(number_of_inputs))
    
    return CircuitQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        sampling=True,
        interpret=interpretation,
        quantum_instance=quantum_instance)

In [16]:
def compare_elements(first, second):
    return first == second

vcompare_elements = np.vectorize(compare_elements)

def is_all_true(arg):
    newarr = []
    for a in arg:
        newarr.append(np.all(a == True))
    return newarr

def score_model(observed, actual_values):
    array_is_correct = is_all_true(
        vcompare_elements(observed, actual_values)
    )
    number_of_correct = array_is_correct.count(True)
    return number_of_correct / len(array_is_correct)

In [17]:
def run(circuit, X, Y, weights):
    out = circuit.forward(X, weights).astype(int)
    result = []
    for value in out:
        _, counts = np.unique(value, return_counts=True)
        ind = np.argmax(counts)
        if ind == 2:
            result.append([0,0,1])
        if ind == 1:
            result.append([0,1,0])
        if ind == 0:
            result.append([1,0,0])
    return result

def run_and_score(circuit, X, Y, weights):
    result = run(circuit, X, Y, weights)
    return score_model(result, Y)

# test_and_score(circuit_qnn, X_test, Y_test, weights_array[-1])

In [18]:
# evaluate data points
def draw_result(circuit, X, Y, weights):    
    Y_predict = run(circuit, X, Y, weights)
    parameter_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    permutations = [[0,1], [0,2], [0,3], [1,2], [1,3], [2,3]]
    fig, axs = plt.subplots(2, 3)
    # plot results
    # red == wrongly classified
    for ind, pair in enumerate(permutations):
        for x, y_target, y_p in zip(X, Y, Y_predict):
            if np.all(y_target == [0,0,1]):
                axs[ind // 3, ind % 3].plot(x[pair[0]], x[pair[1]], 'bo')
            elif np.all(y_target == [0,1,0]):
                axs[ind // 3, ind % 3].plot(x[pair[0]], x[pair[1]], 'yo')
            else:
                axs[ind // 3, ind % 3].plot(x[pair[0]], x[pair[1]], 'go')
            if not np.all(y_target == y_p):
                axs[ind // 3, ind % 3].scatter(x[pair[0]], x[pair[1]], s=200, facecolors='none', edgecolors='r', linewidths=2)

In [22]:
import time

test_cases = [

#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "COBYLA200-ZZFeature1-RealAmplitudes1"],
#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "COBYLA200-ZZFeature2-RealAmplitudes1"],
#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "COBYLA200-ZZFeature4-RealAmplitudes1"],
#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "COBYLA200-ZZFeature1-RealAmplitudes2"],
#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "COBYLA200-ZZFeature1-RealAmplitudes4"],
#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "COBYLA200-ZZFeature2-RealAmplitudes2"],
#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "COBYLA200-ZZFeature2-RealAmplitudes4"],
#     [COBYLA(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "COBYLA200-ZZFeature4-RealAmplitudes4"],
    
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZZFeature1-RealAmplitudes1"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZZFeature2-RealAmplitudes1"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZZFeature4-RealAmplitudes1"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZZFeature1-RealAmplitudes2"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZZFeature2-RealAmplitudes2"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZZFeature4-RealAmplitudes2"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZZFeature1-RealAmplitudes4"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZZFeature2-RealAmplitudes4"],
#     [ADAM(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZZFeature4-RealAmplitudes4"],
    
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZZFeature1-RealAmplitudes1"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZZFeature2-RealAmplitudes1"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZZFeature4-RealAmplitudes1"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZZFeature1-RealAmplitudes2"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZZFeature2-RealAmplitudes2"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZZFeature4-RealAmplitudes2"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "SPSA200-ZZFeature1-RealAmplitudes4"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "SPSA200-ZZFeature2-RealAmplitudes4"],
#     [SPSA(maxiter=200), ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "SPSA200-ZZFeature4-RealAmplitudes4"],
    
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "COBYLA200-ZFeature1-RealAmplitudes1"],
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "COBYLA200-ZFeature2-RealAmplitudes1"],
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "COBYLA200-ZFeature4-RealAmplitudes1"],
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "COBYLA200-ZFeature1-RealAmplitudes2"],
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "COBYLA200-ZFeature1-RealAmplitudes4"],
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "COBYLA200-ZFeature2-RealAmplitudes2"],
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "COBYLA200-ZFeature2-RealAmplitudes4"],
#     [COBYLA(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "COBYLA200-ZFeature4-RealAmplitudes4"],
    
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZFeature1-RealAmplitudes1"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZFeature2-RealAmplitudes1"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZFeature4-RealAmplitudes1"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZFeature1-RealAmplitudes2"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZFeature2-RealAmplitudes2"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZFeature4-RealAmplitudes2"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZFeature1-RealAmplitudes4"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZFeature2-RealAmplitudes4"],
#     [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZFeature4-RealAmplitudes4"],
    
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZFeature1-RealAmplitudes1"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZFeature2-RealAmplitudes1"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZFeature4-RealAmplitudes1"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZFeature1-RealAmplitudes2"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZFeature2-RealAmplitudes2"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZFeature4-RealAmplitudes2"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "SPSA200-ZFeature1-RealAmplitudes4"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "SPSA200-ZFeature2-RealAmplitudes4"],
#     [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "SPSA200-ZFeature4-RealAmplitudes4"],
    
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "SPSA200-PauliFeature1-RealAmplitudes1"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "SPSA200-PauliFeature2-RealAmplitudes1"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "SPSA200-PauliFeature4-RealAmplitudes1"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "SPSA200-PauliFeature1-RealAmplitudes2"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "SPSA200-PauliFeature2-RealAmplitudes2"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "SPSA200-PauliFeature4-RealAmplitudes2"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "SPSA200-PauliFeature1-RealAmplitudes4"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "SPSA200-PauliFeature2-RealAmplitudes4"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "SPSA200-PauliFeature4-RealAmplitudes4"],
    
#     [COBYLA(maxiter=200), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "COBYLA200-PauliFeature1-RealAmplitudes1"],
#     [COBYLA(maxiter=200), PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "COBYLA200-PauliFeature2-RealAmplitudes1"],
#     [COBYLA(maxiter=200), PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "COBYLA200-PauliFeature4-RealAmplitudes1"],
#     [COBYLA(maxiter=50), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "COBYLA50-PauliFeature1-RealAmplitudes2"],
#     [COBYLA(maxiter=200), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "COBYLA200-PauliFeature1-RealAmplitudes4"],
#     [COBYLA(maxiter=200), PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "COBYLA200-PauliFeature2-RealAmplitudes2"],
#     [COBYLA(maxiter=200), PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "COBYLA200-PauliFeature2-RealAmplitudes4"],
#     [COBYLA(maxiter=200), PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "COBYLA200-PauliFeature4-RealAmplitudes4"],

#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZFeature1-RealAmplitudes1"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZFeature2-RealAmplitudes1"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "ADAM200-ZFeature4-RealAmplitudes1"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZFeature1-RealAmplitudes2"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZFeature2-RealAmplitudes2"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "ADAM200-ZFeature4-RealAmplitudes2"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZFeature1-RealAmplitudes4"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZFeature2-RealAmplitudes4"],
#       [ADAM(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "ADAM200-ZFeature4-RealAmplitudes4"],
    
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "ADAM200-PauliFeature1-RealAmplitudes1"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "ADAM200-PauliFeature2-RealAmplitudes1"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1), "ADAM200-PauliFeature4-RealAmplitudes1"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2), "ADAM200-PauliFeature1-RealAmplitudes2"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2), "ADAM200-PauliFeature2-RealAmplitudes2"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "ADAM200-PauliFeature4-RealAmplitudes2"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "ADAM200-PauliFeature1-RealAmplitudes4"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4), "ADAM200-PauliFeature2-RealAmplitudes4"],
#       [ADAM(maxiter=200), PauliFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "ADAM200-PauliFeature4-RealAmplitudes4"],
    
    
    
#     [SPSA(maxiter=2), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA2-ZZFeature1-RealAmplitudes1"],
#       [SPSA(maxiter=1), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZFeature1-RealAmplitudes1"],
#     [SPSA(maxiter=2), PauliFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA2-PauliFeature1-RealAmplitudes1"],
#     [COBYLA(maxiter=120), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "COBYLA120-PauliFeature1-RealAmplitudes2"],
#     [COBYLA(maxiter=120), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "COBYLA120-ZZFeature1-RealAmplitudes4"],
    [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZFeature4-RealAmplitudes2"],
#     [SPSA(maxiter=120), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "SPSA120-ZFeature4-RealAmplitudes4"],
#     [SPSA(maxiter=120), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "SPSA120-ZZFeature2-RealAmplitudes1"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "SPSA200-PauliFeature1-RealAmplitudes1"],
#     [SPSA(maxiter=120), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "SPSA120-PauliFeature1-RealAmplitudes4"],
#     [SLSQP(maxiter=120), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "SLSQP120-PauliFeature1-RealAmplitudes2"],
]

data = []

for test_case_ind, test_case in enumerate(test_cases):
    
    optimizer = test_case[0]
    feature_map = test_case[1]
    ansatz = test_case[2]
    case_name = test_case[3]
    
    print("Test case: ", case_name)

    # construct variational quantum classifier
    circuit_qnn = construct_circuit(feature_map, ansatz, quantum_instance, num_inputs)
#     vqc = NeuralNetworkClassifier(circuit_qnn, loss='cross_entropy', optimizer=optimizer, quantum_instance=quantum_instance, callback=callback_graph)
    vqc = VQC(feature_map=feature_map,
          ansatz=ansatz,
          loss='cross_entropy',
          optimizer=optimizer,
          quantum_instance=quantum_instance,
          callback=callback_graph)
    objective_func_vals = []
    weights_array = []

#     plt.rcParams["figure.figsize"] = (12, 6)

    f = open("objective_func_vals.txt", "a")
    f.write(case_name)
    f.write("\n")
    f.write(str(time.time()))
    f.write("\n")
    f.close()
    
    f = open("weights_array.txt", "a")
    f.write(case_name)
    f.write("\n")
    f.write(str(time.time()))
    f.write("\n")
    f.close()

    # fit classifier to data
    vqc.fit(X_train, Y_train)

    
    f = open("objective_func_vals.txt", "a")
    f.write(str(time.time()))
    f.write("\n")
    f.close()
    
    f = open("weights_array.txt", "a")
    f.write(str(time.time()))
    f.write("\n")
    f.close()
    # return to default figsize
#     plt.rcParams["figure.figsize"] = (6, 4)
    
#     print(vqc.score(X_test, Y_test))
    
    accuracy_for_each_step = []
    
#     circuit_qnn = construct_circuit(feature_map, ansatz, quantum_instance, num_inputs)
    
#     for weight in weights_array:
#         accuracy_for_each_step.append(run_and_score(circuit_qnn, X_test, Y_test, weight))
    
    data.append([case_name, weights_array, objective_func_vals, accuracy_for_each_step])
    
#     print(accuracy_for_each_step)
#     draw_result(circuit_qnn, X_test, Y_test, weights_array[-1])

to_write = pd.DataFrame(data, columns = ['Name', 'Weights Evolution', 'Objective Function Values Evolution', 'Accuracy Evolution'])

Test case:  SPSA200-PauliFeature1-RealAmplitudes1


In [39]:
to_write = pd.DataFrame(data, columns = ['Name', 'Weights Evolution', 'Objective Function Values Evolution', 'Accuracy Evolution'])
to_write.to_csv("Z4Real2-realdevicetraining", index=False)

In [30]:
test_cases = [  
#     [SPSA(maxiter=2), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA2-ZZFeature1-RealAmplitudes1"],
#       [SPSA(maxiter=1), ZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA200-ZFeature1-RealAmplitudes1"],
#     [SPSA(maxiter=2), PauliFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=1), "SPSA2-PauliFeature1-RealAmplitudes1"],
#     [COBYLA(maxiter=120), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "COBYLA120-PauliFeature1-RealAmplitudes2"],
#     [COBYLA(maxiter=120), ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), "COBYLA120-ZZFeature1-RealAmplitudes4"],
    [SPSA(maxiter=200), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), "SPSA200-ZFeature4-RealAmplitudes2"],
#     [SPSA(maxiter=120), ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), "SPSA120-ZFeature4-RealAmplitudes4"],
#     [SPSA(maxiter=120), ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), "SPSA120-ZZFeature2-RealAmplitudes1"],
#     [SPSA(maxiter=200), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), "SPSA200-PauliFeature1-RealAmplitudes1"],
#     [SPSA(maxiter=120), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), "SPSA120-PauliFeature1-RealAmplitudes4"],
#     [SLSQP(maxiter=120), PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), "SLSQP120-PauliFeature1-RealAmplitudes2"],
]

for test_case_ind, test_case in enumerate(test_cases):
    
    optimizer = test_case[0]
    feature_map = test_case[1]
    ansatz = test_case[2]
    case_name = test_case[3]
    accuracy_for_each_step_z = []
    for weight in data[0][1]:
        circuit_qnn = construct_circuit(feature_map, ansatz, quantum_instance, num_inputs)
        accuracy_for_each_step_z.append(run_and_score(circuit_qnn, X_test, Y_test, weight))

In [37]:
print(data2)

[['SPSA200-PauliFeature1-RealAmplitudes1', [array([-0.1466655 ,  0.51177741,  0.92273317,  0.7816575 ,  1.16220922,
        0.63308305,  0.70088542,  0.82196954]), array([0.2533345 , 0.91177741, 0.52273317, 1.1816575 , 0.76220922,
       0.23308305, 1.10088542, 0.42196954]), array([-0.1466655 ,  0.91177741,  0.92273317,  1.1816575 ,  1.16220922,
        0.63308305,  0.70088542,  0.42196954]), array([0.2533345 , 0.51177741, 0.52273317, 0.7816575 , 0.76220922,
       0.23308305, 1.10088542, 0.82196954]), array([-0.1466655 ,  0.51177741,  0.92273317,  1.1816575 ,  1.16220922,
        0.23308305,  1.10088542,  0.82196954]), array([0.2533345 , 0.91177741, 0.52273317, 0.7816575 , 0.76220922,
       0.63308305, 0.70088542, 0.42196954]), array([0.2533345 , 0.51177741, 0.92273317, 0.7816575 , 0.76220922,
       0.63308305, 1.10088542, 0.42196954]), array([-0.1466655 ,  0.91177741,  0.52273317,  1.1816575 ,  1.16220922,
        0.23308305,  0.70088542,  0.82196954]), array([-0.1466655 ,  0.51177